In [ ]:
import yfinance as yf
import pandas as pd

# CPI 데이터 로드
data = pd.read_csv('processed/Approach11.csv', index_col=0, parse_dates=True)

data

,GASREGW,DCOILWTICO,Release,CPI_YOY,WTI+CPI,10MA,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
date,,,,,,,,,,,,,,,,,
2011-05-25,0.324025,1.343837,True,3.077234,3.090673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-26,0.324025,1.343837,False,3.077234,3.090673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-27,0.324025,1.343837,False,3.077234,3.090673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-28,0.324025,1.343837,False,3.077234,3.090673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-29,0.324025,1.343837,False,3.077234,3.090673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-18,-0.095340,-0.174864,False,2.814270,2.812522,2.868423,-0.001723,-0.013982,0.001645,-0.001222,-0.007974,-0.015225,-0.011404,-0.006164,-0.006944,0.000477,-0.017816
2025-03-19,-0.095340,-0.115086,False,2.814270,2.813120,2.849849,0.003337,0.011886,0.016313,0.011014,0.013270,0.013010,0.000000,0.001193,0.003687,0.000545,0.019173
2025-03-20,-0.095340,-0.115086,False,2.814270,2.813120,2.831275,-0.006307,-0.002164,0.003663,0.001412,-0.005014,-0.007119,-0.004890,-0.001191,0.004054,0.000613,-0.002383
